In [2]:
# installs packages into kernel, not environment
import sys
!{sys.executable} -m pip install numpy geopandas pandas matplotlib momepy networkx shapely numpy

import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import momepy
import networkx as nx
import shapely
import numpy as np

import warnings
warnings.filterwarnings("ignore")

  Using cached numpy-1.26.2-cp39-cp39-macosx_10_9_x86_64.whl.metadata (61 kB)
  Using cached geopandas-0.14.1-py3-none-any.whl.metadata (1.5 kB)
  Using cached pandas-2.1.4-cp39-cp39-macosx_10_9_x86_64.whl.metadata (18 kB)
  Using cached momepy-0.6.0-py3-none-any.whl (275 kB)
  Using cached networkx-3.2.1-py3-none-any.whl.metadata (5.2 kB)
  Using cached shapely-2.0.2-cp39-cp39-macosx_10_9_x86_64.whl.metadata (7.0 kB)
  Using cached fiona-1.9.5-cp39-cp39-macosx_10_15_x86_64.whl.metadata (49 kB)
  Using cached pyproj-3.6.1-cp39-cp39-macosx_10_9_x86_64.whl.metadata (31 kB)
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.2/156.2 kB 1.9 MB/s eta 0:00:0000:0100:01
  Using cached libpysal-4.8.1-py3-none-any.whl.metadata (4.6 kB)
  Using cached tqdm-4.66.1-py3-none-any.whl.metadata (57 kB)
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
  Using cached click_plugins-1.1.1-py2.py3-none-any.whl (7.5 kB)
  Using cached clig

/Users/danwillett/anaconda3/envs/connectivity/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
bike_network_path = '../../data/bike_networks/bike_infrastructure_network.shp'
bike_network = gpd.read_file(bike_network_path)

low_stress_path = '../../data/lowstress_SB/lowstress_SB.shp'
low_stress = gpd.read_file(low_stress_path)
low_stress = low_stress.to_crs(bike_network.crs)

all_streets_path = '../../data/all_street_pfb_SB/neighborhood_ways.shp'
all_streets = gpd.read_file(all_streets_path)
all_streets = all_streets.to_crs(bike_network.crs)

block_path = '../../data/20231120_blockgroup/var1120.shp'
blocks = gpd.read_file(block_path)
blocks = blocks.to_crs(bike_network.crs)

In [12]:
# remove multilinestrings from the low_stress dataframe. These are small roads that have been breaks between them, 
# but aren't located on primary roads
def remove_multis(shp_file, multi_output_name):
    geoms = shp_file['geometry']
    geomTypes = pd.Series([type(line) for line in geoms])
    multiTypes = geoms.apply(lambda x: isinstance(x, shapely.geometry.multilinestring.MultiLineString))
    multiLines = shp_file[multiTypes]

    # save multiline polygons to see how many are removed and if they are important
    output_shapefile_path = './outputs/' + multi_output_name
    multiLines.to_file(output_shapefile_path, driver='ESRI Shapefile')
    
    return shp_file[~multiTypes]

# Calculating conductivity indicators
# network density = (total length of bike links)/(block area)
def calc_net_density(bike_edges, polygon):
    total_len = sum(bike_edges['edge_length'])/1000 # covert to km
    total_area = polygon['area'].values[0]/1000000 #convert to km2
    net_density = total_len/total_area # need to figure out what my data layers are
    return net_density

# Gamma Connectivity (degree of connectivity) = edges/lmax, where lmax = 3(n-2)
def calc_gamma(nodes, edges):
    gamma_connectivity = (sum(edges['weights']))/(3*(len(nodes)-2))
    return gamma_connectivity

# Degree of network coverage (Number of bike links)/(number of street links)
def calc_net_coverage(bike_edges, street_edges):
    net_coverage = sum(bike_edges['weights'])/sum(street_edges['weights'])
    return net_coverage

# Intersection density (Number of bike network intersections)/(block area)
def calc_int_density(bike_nodes, polygon):
    total_area = polygon['area'].values[0]/1000000 #convert to km2
    int_density = len(bike_nodes)/total_area
    return int_density

# Degree of network complexity (Number of bike links)/(bike nodes)
def calc_complexity(bike_edges, bike_nodes):
    net_complexity = sum(bike_edges['weights'])/(len(bike_nodes))
    return net_complexity

In [ ]:
# all units in meters (from crs)

# output census block shapefile path with appended connectivity indicator data
output_blocks_path = './outputs/connectivity_blocks.shp'

# list to store connectivity indicators for each block
# b = bike_network, l = low_stress
connectivity_indicators_list = {
        'GEOID': [],
        'b_net_dens': [],
        'ls_net_dens': [],
        'b_gamma': [],
        'ls_gamma': [],
        'b_cover': [],
        'ls_cover': [],
        'b_int_dens': [],
        'ls_int_dens': [],
        'b_complex': [],
        'ls_complex': []
    }

# iterate through each census block, calculating and storing connectivity indicators
for index, row in blocks.iterrows():
        
    row = row.to_frame().T # reshape row to be a dataframe with columns
    id = row['GEOID'].values[0] # store unique GEOID of the current census block
    polygon = gpd.GeoDataFrame(row, geometry='geometry', crs=blocks.crs)
    bike_polylines = remove_multis(bike_network, 'bike')
    low_stress_polylines = remove_multis(low_stress, 'low_stress')
    all_streets_polylines = remove_multis(all_streets, 'all_streets')

    # select networks that reside in current block
    clipped_bike = gpd.sjoin(bike_polylines, polygon, how='inner', op='intersects')
    clipped_low_stress = gpd.sjoin(low_stress_polylines, polygon, how='inner', op='intersects')
    clipped_streets = gpd.sjoin(all_streets_polylines, polygon, how='inner', op='intersects')

    # find the new edge lengths of roads that are cut by block boundaries
    clipped_bike_edge = bike_polylines.clip(polygon)
    clipped_bike['edge_length'] = clipped_bike_edge['geometry'].length
    
    clipped_low_stress_edge = low_stress_polylines.clip(polygon)
    clipped_low_stress['edge_length'] = clipped_low_stress_edge['geometry'].length
    
    clipped_streets_edge = all_streets_polylines.clip(polygon)
    clipped_streets['edge_length'] = clipped_streets_edge['geometry'].length

    # calculate a weighting factor for clipped edges
    clipped_bike['weights'] = clipped_bike['edge_length']/clipped_bike['geometry'].length
    clipped_low_stress['weights'] = clipped_low_stress['edge_length']/clipped_low_stress['geometry'].length
    clipped_streets['weights'] = clipped_streets['edge_length']/clipped_streets['geometry'].length

    # Create a multigraph of nodes and edges where roads intersect
    bike_G = momepy.gdf_to_nx(clipped_bike, approach="primal")
    low_stress_G = momepy.gdf_to_nx(clipped_low_stress, approach="primal")
    street_G = momepy.gdf_to_nx(clipped_streets, approach="primal")
    
    # Getting nodes and edges
    bike_nodes, bike_edges, bike_weights = momepy.nx_to_gdf(bike_G, spatial_weights=True)
    low_stress_nodes, low_stress_edges, low_stress_weights = momepy.nx_to_gdf(low_stress_G, spatial_weights=True)
    street_nodes, street_edges, street_weights = momepy.nx_to_gdf(street_G, spatial_weights=True)
    
    # calculating connectivity indicators
    bike_network_density = calc_net_density(bike_edges, polygon)
    low_stress_network_density = calc_net_density(low_stress_edges, polygon)

    bike_gamma = calc_gamma(bike_nodes, street_edges)
    low_stress_gamma = calc_gamma(low_stress_nodes, street_edges)

    bike_network_coverage = calc_net_coverage(bike_edges, street_edges)
    low_stress_network_coverage = calc_net_coverage(low_stress_edges, street_edges)

    bike_intersection_density = calc_int_density(bike_nodes, polygon)
    low_stress_intersection_density = calc_int_density(low_stress_nodes, polygon)
    
    bike_network_complexity = calc_complexity(bike_edges, bike_nodes)
    low_stress_network_complexity = calc_complexity(low_stress_edges, low_stress_nodes)
    
    # store connectivity indicators in list
    connectivity_indicators_list['GEOID'].append(id)
    
    connectivity_indicators_list['b_net_dens'].append(bike_network_density)
    connectivity_indicators_list['ls_net_dens'].append(low_stress_network_density)
    
    connectivity_indicators_list['b_gamma'].append(bike_gamma)
    connectivity_indicators_list['ls_gamma'].append(low_stress_gamma)
    
    connectivity_indicators_list['b_cover'].append(bike_network_coverage)
    connectivity_indicators_list['ls_cover'].append(low_stress_network_coverage)

    connectivity_indicators_list['b_int_dens'].append(bike_intersection_density)
    connectivity_indicators_list['ls_int_dens'].append(low_stress_intersection_density)

    connectivity_indicators_list['b_complex'].append(bike_network_complexity)
    connectivity_indicators_list['ls_complex'].append(low_stress_network_complexity)
    
# Convert the list to a new DataFrame
connectivity_indicators_df = pd.DataFrame.from_dict(connectivity_indicators_list)
print(connectivity_indicators_df.head())

# Merge dataframe with blocks geodataframe on GEOID attribute
connectivity_blocks = blocks.merge(connectivity_indicators_df, on='GEOID')

# save updated census blocks to new file
connectivity_blocks.to_file(output_blocks_path, driver='ESRI Shapefile', if_exists='replace')

In [ ]:
# positions = {n: [n[0], n[1]] for n in list(G.nodes)}

# # Plot
# f, ax = plt.subplots(1, 2, figsize=(12, 6), sharex=True, sharey=True)
# low_stress_polylines.plot(color="k", ax=ax[0])
# for i, facet in enumerate(ax):
#     facet.set_title(("Streets", "Graph")[i])
#     facet.axis("off")
# nx.draw(G, positions, ax=ax[1], node_size=5)

: 